회귀분석을 이용한 동 별 반려동물 등록 수 예측 모델
- 독립변수: 동 별 인구 수
- 종속변수: 동 별 반려동물 등록 수 

In [42]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression

import pandas as pd
import numpy as np

In [43]:
concat_original_registrations = {'Dong': ['쌍문동','방학동','도봉통','창동','공릉동','상계동','월계동','중계동','하계동'],
                                 'total': [3717,3481,2081,5397,4586,11091,4227,5849,2376],
                                 'population': [75188,74624,46981,117556,79722,199483,29384,103155,48624]}

concat_original_registrations = pd.DataFrame(concat_original_registrations)
concat_original_registrations.columns = ['Dong','total','population']

x = concat_original_registrations[['population']]
y = concat_original_registrations['total']

model = LinearRegression()
result = model.fit(x, y)

In [44]:
# 유의성 검정
import statsmodels.api as sm

x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
f_statistic = model.fvalue
p_value = model.f_pvalue

print("F-statistic:", f_statistic)
print("p-value:", p_value)

F-statistic: 42.1474868870474
p-value: 0.00033662313056861227


In [45]:
# 예측하고자 하는 데이터 가져오기

In [46]:
# 서대문구 동 별 행 
Seodaemun_population = {'Dong': ['천연동', '북아현동', '충현동', '신촌동', '연희동', '홍제동', '홍은동', '남가좌동', '북가좌동'],
                        'population': [16678, 15549, 19757, 20322, 34202, 55835, 51107, 45962, 48458]}
Seodaemun_population = pd.DataFrame(Seodaemun_population['population'])

total_registration_seodaemun = 16944 

Seodaemun_pet_registration = result.predict(Seodaemun_population)
Seodaemun_pet_registration = pd.DataFrame(Seodaemun_pet_registration)

print(Seodaemun_pet_registration)
print()
print('서대문 구의 총 동물 등록 수: ',Seodaemun_pet_registration.sum())

             0
0  1356.745845
1  1301.446259
2  1507.558443
3  1535.232726
4  2215.089452
5  3274.696121
6  3043.113801
7  2791.106390
8  2913.363046

서대문 구의 총 동물 등록 수:  0    19938.352082
dtype: float64


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


모델을 사용하여, 동 별 반려동물 등록자 수를 측정한 후, 실제 소유자 수와 비교

In [55]:
# 동 별 동물 등록 현황 실 데이터
original = {'Dong': ['쌍문동','방학동','도봉통','창동','공릉동','상계동','월계동','중계동','하계동'],
            'registration': [3717,3481,2081,5397,4586,11091,4227,5849,2376]}

original = pd.DataFrame(original)

original_registrations = original['registration']


# 여러 구 인구수 데이터(예측에 이용)
concat_population = {'Dong': ['쌍문동','방학동','도봉통','창동','공릉동','상계동','월계동','중계동','하계동'],
                     'population': [75188,74624,46981,117556,79722,199483,29384,103155,48624]}

concat_population = pd.DataFrame(concat_population)

# 총 동물 등록 수(예측에 이용)
total_registration = original_registrations.sum()

# 모델을 이용하여 동 별 등록 동물 수 예측

x = concat_population[['population']]
y = original['registration']

model = LinearRegression()
result = model.fit(x, y)
predict_registration = result.predict(x)
#predict_registration = pd.DataFrame(predict_registration)


accuracy_list = []
for i in range(len(predict_registration)):
        print(f"{original['Dong'][i]}")
        print(f'실제 동 별 동물 등록 현황 :{original_registrations[i]}')
        print(f'추정된 소유자 수:{predict_registration[i]}')
        print(f'예측의 정확도:{100*(1- abs(original_registrations[i] - predict_registration[i])/original_registrations[i]):.2f}%')
        accuracy = 100*(1- abs(original_registrations[i] - predict_registration[i])/original_registrations[i])
        accuracy_list.append(accuracy)
    
    
    
print('-'*30)
total_accuracy = sum(accuracy_list)/len(accuracy_list)

print(f"총 정확도: {total_accuracy:.1f}%")


쌍문동
실제 동 별 동물 등록 현황 :3717
추정된 소유자 수:4222.62603590066
예측의 정확도:86.40%
방학동
실제 동 별 동물 등록 현황 :3481
추정된 소유자 수:4195.000733765815
예측의 정확도:79.49%
도봉통
실제 동 별 동물 등록 현황 :2081
추정된 소유자 수:2841.018061933298
예측의 정확도:63.48%
창동
실제 동 별 동물 등록 현황 :5397
추정된 소유자 수:6297.854406200558
예측의 정확도:83.31%
공릉동
실제 동 별 동물 등록 현황 :4586
추정된 소유자 수:4444.706035686813
예측의 정확도:96.92%
상계동
실제 동 별 동물 등록 현황 :11091
추정된 소유자 수:10310.723427479817
예측의 정확도:92.96%
월계동
실제 동 별 동물 등록 현황 :4227
추정된 소유자 수:1979.098839119682
예측의 정확도:46.82%
중계동
실제 동 별 동물 등록 현황 :5849
추정된 소유자 수:5592.478562151066
예측의 정확도:95.61%
하계동
실제 동 별 동물 등록 현황 :2376
추정된 소유자 수:2921.49389776229
예측의 정확도:77.04%
------------------------------
총 정확도: 80.2%


비율을 직접 구하여 동물 등록 수 예측

In [ ]:
# 서대문구 동 별 행 
Seodaemun_population = {'Dong': ['천연동', '북아현동', '충현동', '신촌동', '연희동', '홍제동', '홍은동', '남가좌동', '북가좌동'],
                        'population': [16678, 15549, 19757, 20322, 34202, 55835, 51107, 45962, 48458]}
Seodaemun_population = pd.DataFrame(Seodaemun_population)

# 동 별 인구 수 비율 
Seodaemun_population['ratio'] = Seodaemun_population['population'] / Seodaemun_population['population'].sum()

# 구의 총 반려동물 등록 수 와 각 동별 인구 수 비율을 곱하여 
# 동별 동물 등록 수를 예측

total_registration = 16944
Seodaemun_population['total'] = round(Seodaemun_population['ratio'] * total_registration)
# 16944 서대문구 전체 등록 수 
print(Seodaemun_population)
print(Seodaemun_population['total'].sum())

   Dong  population     ratio   total
0   천연동       16678  0.054172   918.0
1  북아현동       15549  0.050505   856.0
2   충현동       19757  0.064173  1087.0
3   신촌동       20322  0.066008  1118.0
4   연희동       34202  0.111092  1882.0
5   홍제동       55835  0.181359  3073.0
6   홍은동       51107  0.166002  2813.0
7  남가좌동       45962  0.149290  2530.0
8  북가좌동       48458  0.157398  2667.0
16944.0


동별 인구 수 비율을 이용하여 동 별 동물 등록 수를 예측하고, 정확도 검사 

In [ ]:
# 인구 수를 이용한 정확도 예측

In [ ]:
# 동 별 동물 등록 현황 실 데이터
original_registrations = {'Dong': ['쌍문동','방학동','도봉통','창동','공릉동','상계동','월계동','중계동','하계동'],
                          'registration': [3717,3481,2081,5397,4586,11091,4227,5849,2376]}

original_registrations = pd.DataFrame(original_registrations)


# 여러 구 인구수 데이터
concat_population = {'Dong': ['쌍문동','방학동','도봉통','창동','공릉동','상계동','월계동','중계동','하계동'],
                     'population': [75188,74624,46981,117556,79722,199483,29384,103155,48624]}


concat_population = pd.DataFrame(concat_population)

# 동별 인구 수 비율
total_population = concat_population['population'].sum()
concat_population['ratio'] = concat_population['population']/total_population

# 동별 비율에 따른 동물 등록 현황 예측
total_registration = original_registrations['registration'].sum()
concat_population['registration'] = round(concat_population['ratio'] * total_registration,0)



accuracy_list = []
for i in range(len(original_registrations['Dong'])):
    print(f'{original_registrations["Dong"][i]}')
    print(f'실제 동별 동물 등록 현황: {original_registrations["registration"][i]}')
    print(f'예측 동별 동물 등록 현황: {concat_population["registration"][i]}')
    print(f'예측의 정확도:{100*(1- abs(concat_population["registration"][i] - original_registrations["registration"][i])/original_registrations["registration"][i]):.2f}%')
    accuracy = 100*(1- abs(concat_population["registration"][i] - original_registrations["registration"][i])/original_registrations["registration"][i])
    accuracy_list.append(accuracy)
    
    
total_accuracy = sum(accuracy_list)/len(accuracy_list)

print(f"총 정확도: {total_accuracy:.2f}%")

쌍문동
실제 동별 동물 등록 현황: 3717
예측 동별 동물 등록 현황: 4154.0
예측의 정확도:88.24%
방학동
실제 동별 동물 등록 현황: 3481
예측 동별 동물 등록 현황: 4123.0
예측의 정확도:81.56%
도봉통
실제 동별 동물 등록 현황: 2081
예측 동별 동물 등록 현황: 2596.0
예측의 정확도:75.25%
창동
실제 동별 동물 등록 현황: 5397
예측 동별 동물 등록 현황: 6495.0
예측의 정확도:79.66%
공릉동
실제 동별 동물 등록 현황: 4586
예측 동별 동물 등록 현황: 4405.0
예측의 정확도:96.05%
상계동
실제 동별 동물 등록 현황: 11091
예측 동별 동물 등록 현황: 11022.0
예측의 정확도:99.38%
월계동
실제 동별 동물 등록 현황: 4227
예측 동별 동물 등록 현황: 1624.0
예측의 정확도:38.42%
중계동
실제 동별 동물 등록 현황: 5849
예측 동별 동물 등록 현황: 5700.0
예측의 정확도:97.45%
하계동
실제 동별 동물 등록 현황: 2376
예측 동별 동물 등록 현황: 2687.0
예측의 정확도:86.91%
총 정확도: 82.55%


실제 다른 데이터에도 